# Section1：勾配消失問題

誤差逆伝播法が下位層に進んでいくに連れて、勾配がどんどん緩やかになっていく。そのため、勾配降下法による、更新では下位層のパラメータはほとんど変わらず、訓練は最適値に収束しなくなる。

### 活性化関数: シグモイド関数
- 0～1の間を緩やかに変化する関数で、ステップ関数ではON/OFFしかない状態に対し、信号の強弱を伝えられるようになり、予想ニューラルネットワーク普及のきっかけとなった。

#### 課題
- 大きな値では出力の変化が微小なため、勾配消失問題を引き起こす事があった。

## 勾配消失問題のビジョン

## 勾配消失の解決方
- 活性化関数の選択
- 重みの初期値設定
- バッチ正規化

### 活性化関数の選択

#### 活性化関数: RELU関数
- 今最も使われている活性化関数勾配消失問題の回避とスパース化に貢献することで良い成果をもたらしている。

### 重みの初期値設定 Xavier
#### Xavierの初期値を設定する際の活性化関数
- ReLU関数
- シグモイド関数
- 双曲線正接関数

#### 設定方法
- 重みの要素を、前の層のノード数の平方根で除算した値

## 重みの初期値設定 He
#### Heの初期値を設定する際の活性化関数
- ReLU関数

#### 設定方法
- 重みの要素を、前の層のノード数の平方根で除算した値に対し√２をかけ合わせた値

### バッチ正規化

- バッチ正規化とは︖
 - ミニバッチ単位で、入力値のデータの偏りを抑制する手法

- バッチ正規化の使い所とは︖
 - 活性化関数に値を渡す前後に、バッチ正規化の処理を孕んだ層を加える

# 確認テスト

#### P10

問） \
連鎖律の原理を使い、dz/dxを求めよ。 \
z = t2 \
t = x +y10 

答）
\begin{aligned}
\frac{\partial z}{\partial t} &= 2t \\[6px]
\frac{\partial t}{\partial x} &= 1 \\[6px]
\frac{\partial z}{\partial t} \frac{\partial t}{\partial x} &= 2t \times 1 \\[6px]
&= 2t
\end{aligned}

#### P18

問）
シグモイド関数を微分した時、入力値が0の時に最大値をとる。その値として正しいものを選択肢から選べ。 \
（1）0.15 \
（2）0.25 \
（3）0.35 \
（4）0.45

答）（2）0.25

シグモイド関数
\begin{aligned}
y = \frac{1}{1 \; + \; e^{-x}}
\end{aligned}

微分しやすいように式を変換する。
\begin{aligned}
u &= 1 \; + \; e^{-x}  \\[6px]
y &= u^{-1}
\end{aligned}

微分する
\begin{aligned}
\frac{\partial u}{\partial x} &=  e^{-x}  \\[6px]
\frac{\partial y}{\partial u} &= -u^{-2}
\end{aligned}

連鎖律
\begin{aligned}
\frac{\partial y}{\partial x} &= \frac{\partial y}{\partial u} \cdot \frac{\partial u}{\partial x} \\[8px]
&= (-u^{-2}) (-e^{-x})  \\[6px]
&= \frac{e^{-x}}{(1 \; + \; e^{-x})^2} \\[6px]
&=   \frac{ e^{-x}}{1 \; + \; e^{-x}} \cdot \frac{1}{1 \; + \; e^{-x}} 
\end{aligned}

右辺の左側を変形する。
\begin{aligned}
\frac{\partial y}{\partial x} &= \left( \frac{1 \; + \; e^{-x}}{1 \; + \; e^{-x}} - \frac{1}{1 \; + \; e^{-x}} \right) \frac{1}{1 \; + \; e^{-x}} \\[6px]
&= (1 - y) \cdot y
\end{aligned}

In [26]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# シグモイド関数に０を渡した結果
y = sigmoid(0)
print(y)

0.5


In [23]:
# シグモイド関数の導関数を実行する
print((1 - y) * y)

0.25


#### P26

問）
重みの初期値に0を設定すると、どのような問題が発生するか。簡潔に説明せよ。

答） \
■■■■■

#### P29

問）一般的に考えられるバッチ正規化の効果を2点挙げよ。

答） \
■■■■■